    from IPython import get_ipython; ip = get_ipython()
    %load_ext pidgin.post_run_cell
    
`import pidgin.post_run_cell` adds extra steps on `'post_run_cell' in ip.events.callbacks`. After the normal 
`ip.run_cell` evaluation:
    
1. `import doctest`s are executed.
2. Inline code statements and expressions are `exec`uted.
3. The raw source entered into `ip.run_cell` if it should be displayed.

    `import pidgin.display` controls the display behaviors and `import pidgin.tangle` controls
    how the __Markdown__ source is converted to __Python__.

In [2]:
    import IPython, functools, types, textwrap, ast, doctest, sys, traceback
    with __import__('importnb').Notebook():
        try: 
            from .tangle import markdown_to_python, markdown_to_statements
            from . import doctesting
        except: 
            from tangle import markdown_to_python, markdown_to_statements
            import doctesting

1. `run_cell_doctests` _will discover any doctests in the cell and evaluate them_.
2. `run_user_expressions` _`exec`ute inline code statements and expressions to assure that it is valid code_.
3. `run_cell_display` _display the input_.

`show_error` decorates a `post_run_cell` `callable` to show an `Exception` in the result.

In [3]:
    def show_error(ip, object):
        @functools.wraps(object)
        def post_run_cell(result: "Return from `ip.run_cell`"):
            if result.error_in_exec or result.error_before_exec: return result
            result.error_in_exec = object(result)
            if result.error_in_exec:
                ip.showtraceback(exc_to_tuple(result.error_in_exec))
            return result
        return post_run_cell

`exc_to_tuple` converts an `Exception` to a `tuple` to display the traceback__.

In [4]:
    def exc_to_tuple(Exception): return type(Exception), Exception, Exception.__traceback__

`run_cell_doctests` runs any `doctest`s found in the raw source.

In [15]:
    def run_cell_doctests(ip, result) -> Exception:
        with IPython.utils.capture.capture_output(
            stdout=True, stderr=False, display=False
        ) as output:
            doctesting.run_docstring_examples(result.info.raw_cell, ip)

        if output.stdout:
            return DocTestException(output.stdout)

In [16]:
    class DocTestException(BaseException):
        def _render_traceback_(self): return args[0].splitlines()

`run_user_statements` `exec`utes each inline 

In [7]:
    def run_user_statements(ip, result):
        try: user_statements(ip, {str: str for str in markdown_to_statements(result.info.raw_cell)})
        except BaseException as Exception: return Exception

In [8]:
    def user_statements(ip, statements, *, buffer=''):
        """User expressions evaluate expressions, assert statements are the only exception. `assert` may
        augment a literate narrative so we choose to include these statements in the expressions.
        """
        for key, statement in statements.items():
            buffer = buffer.rstrip('\\') + statement  
            if buffer.endswith('\\') and "User statements can continue with a \\": continue
                
            buffer = ip.input_transformer_manager.transform_cell(textwrap.indent(buffer, ' '*4))
            try: exec(buffer, ip.user_global_ns, ip.user_ns)
            except BaseException as Exception: raise InlineCodeException(buffer)
            finally: buffer = ''
        if buffer:
            raise SyntaxError("The final inline code statements may not terminate in a line continuation.")

In [9]:
    class InlineCodeException(BaseException): ...

`run_cell_display` triggers a display on the input if contains anything other than block code.

In [17]:
    def run_cell_display(ip, result) -> bool:
        raw, source = result.info.raw_cell, markdown_to_python(result.info.raw_cell)
        try:
            is_display = bool(raw.partition('\n')[0].strip() and not raw.startswith(';')) 
            if raw.strip() == ast.literal_eval(source.rstrip().rstrip(';')).strip():
                is_display = False
        except: ...
        if is_display and not only_contains_block_code(raw, source):
            try: _exc = ip._get_exc_info()[1]
            except: _exc = None
            IPython.display.display(raw)
            try:
                exc = ip._get_exc_info()[1]
                if _exc is not exc: return exc
            except: ...

`only_contains_block_code` determines if the input source and python only include block code.

In [11]:
    def only_contains_block_code(raw: str, source: str) -> bool: 
        return list(
            map(str.rstrip, textwrap.dedent(raw).splitlines())
        ) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))

# Extensions

In [1]:
    def load_ipython_extension(ip): 
        _show_error = types.MethodType(show_error, ip)
        for callable in (run_user_statements, run_cell_doctests, run_cell_display):
            ip.events.register('post_run_cell', _show_error(types.MethodType(callable, ip)))

    def unload_ipython_extension(ip): 
        ip.events.callbacks['post_run_cell'] = [
            object for object in ip.events.callbacks['post_run_cell']
            if object.__name__ not in {
                'run_cell_doctests', 'run_user_statements', 'run_cell_display'}]
        
    if __name__ == '__main__': 
        %reload_ext pidgin.tangle
        %reload_ext pidgin.display
        unload_ipython_extension(IPython.get_ipython())
        load_ipython_extension(IPython.get_ipython())

NameError: name 'IPython' is not defined